
# Hterm Output TDC Hamming Weight to Plot Data

This notebook processes space-separated binary data (captured from Hterm) and:
- Groups them into 12-byte sequences
- Reverses each byte's bits
- Calculates the Hamming Weight (total number of 1s per group)
- Outputs a summary per clock cycle

The result is saved as a CSV file for plotting or further analysis.


In [ ]:

import csv
from itertools import islice

# Input/output file paths
bitstream_file_path = 'bitstream_input.txt'       # Input file from Hterm with space-separated binary bytes
output_csv_path = 'bitstream_processed_out.csv'   # Output CSV to write results


In [ ]:

def batched(iterable, n):
    """
    Custom batching function to split the input list into chunks of size n.
    Uses islice to efficiently yield one batch at a time.
    """
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch


In [ ]:

data = []            # This will hold dictionaries of processed results
addressNo = 1        # Counter to track clock cycles (each group of 12 bytes is one clock cycle)

# Read and process the bitstream input file
with open(bitstream_file_path, 'r') as bitstream_file:
    for line in bitstream_file:
        # Split each line into a list of individual 8-bit binary byte strings
        byte_list = line.strip().split()
        
        # Group every 12 bytes into a single "clock cycle" unit
        for address in batched(byte_list, 12):
            processedBytes = []   # Will store reversed versions of the 12 bytes
            count = 0             # Counter for Hamming Weight (total number of '1's in this group)

            for b in address:
                reversed_b = b[::-1]              # Reverse bits in each byte (e.g., '10110000' → '00001101')
                processedBytes.append(reversed_b) # Add to processed list
                count += reversed_b.count('1')    # Count '1's in the reversed byte and add to total
            
            # Append the processed result to the data list
            data.append({
                'Clk Cycle': addressNo,
                'Bytes': ' '.join(processedBytes),     # Concatenate all 12 reversed bytes into one string
                'Hamming Weight': count                 # Total '1's in the 12 reversed bytes
            })
            addressNo += 1  # Move to the next clock cycle


In [ ]:

# Write the processed data to a CSV file
with open(output_csv_path, 'w', newline='') as output_file:
    # Define CSV column headers
    writer = csv.DictWriter(output_file, fieldnames=['Clk Cycle', 'Bytes', 'Hamming Weight'])
    writer.writeheader()      # Write the header to CSV
    writer.writerows(data)    # Write each processed row

print(f"Saved output to {output_csv_path}")
